# Data Exploration - Phase II
Shpaner, Leonid  
Robinson, Christopher  
Luis-Estrada, Jose  

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/Shared drives/Capstone - Best Group/navigating_crime/Code Library

/content/drive/Shared drives/Capstone - Best Group/navigating_crime/Code Library


In [3]:
####################################
## import the requisite libraries ##
####################################
import os
import csv
import pandas as pd
import numpy as np

In [4]:
# check current working directory
current_directory = os.getcwd()
current_directory

'/content/drive/Shared drives/Capstone - Best Group/navigating_crime/Code Library'

### Assign Paths to Folders

In [5]:
# path to the data file
data_path1 = '/content/drive/Shareddrives/Capstone - Best Group/' \
           + 'Final_Data_20220719/CrimesLA_Streets_with_Crimes.csv'

# path to data folder
data_path2 = '/content/drive/Shareddrives/Capstone - Best Group/navigating_crime/'\
             'Data Folder/'

# path to the image library
image_path = '/content/drive/Shareddrives/Capstone - Best Group/navigating_crime'\
          + '/Image Folder'        

In [8]:
# read in the training .csv file to a dataframe using pandas
df_ml = pd.read_csv(data_path2 + 'train_set.csv', low_memory=False)
df_ml.head()

,OBJECTID,Zip_Code,Accuracy,Crime_Cost,StreetOID,DR_NO,TIME_OCC,AREA,Rpt_Dist_No,Part_1_2,Vict_Age,Premis_Cd,Weapon_Used_Cd,Days_Disclosure,Street_Type,Gender,Crime_Code
0,69772,90024.0,50.0,2.0,27815,210814900.0,750.0,8.0,833.0,2.0,40.0,102.0,400.0,0,7,1,0
1,131641,90037.0,50.0,3.0,51117,210313311.0,1100.0,3.0,397.0,1.0,32.0,101.0,106.0,0,8,2,0
2,143866,90016.0,50.0,1.0,55526,200705585.0,1420.0,7.0,785.0,1.0,46.0,101.0,102.0,0,7,2,1
3,189323,90003.0,50.0,0.0,72325,221213641.0,910.0,12.0,1268.0,2.0,25.0,102.0,400.0,0,8,1,0
4,89956,91331.0,50.0,2.0,35427,221606939.0,1625.0,16.0,1612.0,2.0,63.0,108.0,400.0,0,6,2,0


In [9]:
# show the columns of the dataframe for inspection
df_ml.columns

Index(['OBJECTID', 'Zip_Code', 'Accuracy', 'Crime_Cost', 'StreetOID', 'DR_NO',
       'TIME_OCC', 'AREA', 'Rpt_Dist_No', 'Part_1_2', 'Vict_Age', 'Premis_Cd',
       'Weapon_Used_Cd', 'Days_Disclosure', 'Street_Type', 'Gender',
       'Crime_Code'],
      dtype='object')

In [10]:
# re-inspect the shape of the dataframe. This is also done on EDA file.
df_ml.shape

(128205, 17)

In [ ]:
# rename Zip_R to Zip_Code, since this will be the only column used for zip code
df = df.rename(columns={'Zip_R': 'Zip_Code'})

In [ ]:
# run the python file created for showing columns, data types, and their
# respective null counts
%run data_types.py

In [ ]:
data_types = data_types(df)

In [ ]:
pd.set_option('display.max_rows', None)
data_types = data_types.sort_values(by='# of Nulls', ascending=False)

In [ ]:
# here only the top null columns are subset
# a good amount of these columns are to be subsequently removed
dat_typ = data_types[(data_types['Percent Null']<84) & \
                     (data_types['# of Nulls']>0)]

# any column with '_L' and '_R' is of no interest and will be dropped
dat_typ = dat_typ[dat_typ['Column/Variable'].str.contains('_L')==False]
dat_typ = dat_typ[dat_typ['Column/Variable'].str.contains('_R')==False]
dat_typ

# object types are typically removed for ML processing, but some imputation
# may be necessary, which is why they are examined prior

,Column/Variable,Data Type,# of Nulls,Percent Null
16,StPreDir,object,83241,45.0
67,Cross_Street,object,74575,41.0
21,StPostType,object,6189,3.0
58,Weapon_Used_Cd,float64,3274,2.0
59,Weapon_Desc,object,3274,2.0
20,StName,object,1357,1.0
4,FullName,object,1357,1.0
52,Mocodes,object,219,0.0
25,Zip_Code,float64,211,0.0
55,Vict_Descent,object,171,0.0


For example, there are 3,274 missing valuesd for 'Weapon_Desc.' Imputation is necessary to properly assign the corresponding numerical value to the corresponding 'Weapon_Used_Cd,' which is also missing 3,274 observations.  

The 'Vict_Descent' column only has 171 missing values, the rows corresponding to these values can be dropped, but they are instead imputed with 'Unknown' since the ethnicities here remain unknown.  

The same holds true for 'Vict_Sex,' where there exist 167 missing values. 

Ethnicity and age are important features which will be numericized and adapted for use with machine learning. Jurisdiction and other string columns presented herein will be subsequently omitted from the development (modeling) set.

## Imputation Strategies

In [ ]:
# if the weapon used is unknown/other weapon, find the associated code with for
# that type of weapon
df_weapon = df[(df['Weapon_Desc'] == 'UNKNOWN WEAPON/OTHER WEAPON') & \
               (df['Weapon_Used_Cd']>0)]
df_weapon['Weapon_Used_Cd'].head()

OBJECTID
18     500.0
61     500.0
69     500.0
116    500.0
151    500.0
Name: Weapon_Used_Cd, dtype: float64

In [ ]:
# Since the code for unknown weapons is 500, missing values for 'Weapon_Used_Cd' 
# where 'Weapon_Desc' is 'UNKNOWN WEAPON/OTHER WEAPON' will be 500.
# However, since missing values in one column correspond to the other, 
# missing values in 'Weapon_Desc' will need to be imputed first with
# 'UNKNOWN WEAPON/OTHER WEAPON'

# impute missing weapon descriptions
df['Weapon_Desc'] = df['Weapon_Desc'].fillna('UNKNOWN WEAPON/OTHER WEAPON')

# impute missing 'Weapon_Used_Cd' with 500 for reasons discussed above.
df['Weapon_Used_Cd'] = df['Weapon_Used_Cd'].fillna(500)

# impute missing values for victim's gender with 'X,' since this corresponds to
# 'Unknown' in the data dictionary
df['Vict_Sex'] = df['Vict_Sex'].fillna('X')

# impute missing values for victim's race with 'X,' since this corresponds to
# 'Unknown' in the data dictionary
df['Vict_Descent'] = df['Vict_Descent'].fillna('X')

# Since there are only 211 rows with missing zip codes, those rows are dropped
df.dropna(subset=['Zip_Code'], inplace=True)

## Feature Engineering

In [ ]:
# converting dates to datetime format so that they are not cast as objects
df['Date_Disclosed'] = pd.to_datetime(df['Date_Rptd'])
df['Date_Occurred'] = pd.to_datetime(df['DATE_OCC'])

# taking the difference of the two datetime objects as number of days between
# disclosure and occurrence
df['Days_Disclosure'] = (df['Date_Disclosed'] 
                      -  df['Date_Occurred']).dt.days.astype(int)

## Reclassifying Useful Categorical Features

In [ ]:
# numericize 'Type' column to reflect numeric severity scores ranging from 1-8.
df['Street_Type'] = df['Type'].map({'Minor': 8, 'Alley': 5, 'Private Road': 3,
                                    'Primary': 7, 'Secondary': 6, 'Trail': 1, 
                                    'Pedestrian Walkway': 4, 'Unpaved Road': 2
                                    }
                                   )

# narrow down victim sex column to three categories (M, F, X=Unknown). Currently,
# there are four, with "H" presented. "H" is re-categorized to "X" since it is not
# in the data dictionary, and thus, remains unknown.
df['Victim_Sex'] = df['Vict_Sex'].map({'F':'F', 'H':'X', 'M':'M', 'X':'X'})
df['Gender'] = df['Victim_Sex'].map({'F':1, 'M':2, 'X':3})

# reclassify letters in victim description back to full race description from
# data dictionary nomenclature and add as new column; this will be helpful for 
# additional EDA post pre-processing.
df['Victim_Desc'] = df['Vict_Descent'].map({'A':'Other Asian', 'B':'Black', 'C':
                                         'Chinese', 'D': 'Cambodian', 'F':
                                         'Filipino', 'G': 'Guamanian', 'H':
                                         'Hispanic/Latin/Mexican', 'I':
                                         'American Indian/Alaskan Native', 'J':
                                         'Japanese', 'K': 'Korean', 'L': 'Laotian',
                                         'O': 'Other', 'P': 'Pacific Islander',
                                         'S': 'Samoan', 'U': 'Hawaiian', 'V':
                                         'Vietnamese', 'W': 'White', 'X': 'Uknown',
                                         'Z': 'Asian Indian'
                                         }
                                         )
# binarize the 'Crm_Cd' column; this is the target column
# 0 is for any crime that ranges 1-199 (most severe)
# anything above 500 is less severe
df['Crime_Code'] = df['Crm_Cd'].apply(lambda value: '1' if value <= 499 else \
                                      ('0' if value >= 500 else '2'))

## Remove Columns Not Used for Model Development

* Free text  
* All nulls  

In [ ]:
# make a unique list of cols to remove that are completely missing
cols_to_remove_null = data_types[(data_types["Percent Null"] == 100)]\
                                ['Column/Variable'].unique()

# make a unique list of cols to remove that are non-numeric
cols_to_remove_string = data_types[(data_types["Data Type"] == 'object')]\
                                  ['Column/Variable'].unique()

intersect = np.intersect1d(cols_to_remove_null, cols_to_remove_string)
print('The following columns intersect as being objects and being null:',
      intersect, '\n')

# Since these columns exist as objects too, they can be removed from the
# list of nulls such that they are not doubled up between the two.'
refined_null = [x for x in cols_to_remove_null if x not in cols_to_remove_string]

# set-up new df without dropped columns (dataframe used later for modeling)
df_prep = df.drop(columns=refined_null) 

print('Removed Fully Null columns:')
for c in cols_to_remove_null:
    print(f'\t{c}')
print()

# these columns are to be removed b/c they are object datatypes 
# (i.e., free text, strings)
df_prep = df_prep.drop(columns=cols_to_remove_string)

print('Removed String columns:')
for c in cols_to_remove_string:
  print(f'\t{c}')

# drop any column that contains Master Street Address Guide (MSAG)
# since this is tied to specific address and will not be used for predictive
 #modeling

for col in df_prep.columns:
    if 'MSAG' in col:
        df_prep = df_prep.drop(columns=col)
        print('Removed MSAG Columns:')
        print(f'\t{col}')

# drop latitude & longitude columns; they will not be used to inform ML model(s)
print()
print('Removed Latitude and Longtitude Columns')
for col in df_prep.columns:
    if 'LAT' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')
    elif 'LON' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')
# drop any additional crime code columns, since only Crm_Cd without number
# following it will be used as target
print()
print('Removed Extraneous Crm_Cd Columns:')
for col in df_prep.columns:
    if 'Crm_Cd' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')
print()

# removed date columns because date difference already feature engineered
print('Removed Date Columns:')
for col in df_prep.columns:
    if 'Date' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')

# removed victim sex column because numericized as new 'Gender' column
# removed ethnicity since these cannot be numerically classified without bias
print('Removed Victim Sex and Ethnicity Columns:')
for col in df_prep.columns:
  if 'Sex' in col:
      df_prep = df_prep.drop(columns=col)
      print(f'\t{col}') 
  elif 'Desc' in col:
      df_prep = df_prep.drop(columns=col)
      print(f'\t{col}') 

# drop any columns with '_L' or '_R' nomenclature since these are positional
# street indexes which are used to compile other location-based information
print()
print("Removed '_L' and '_R' Columns:")
for col in df_prep.columns:
    if '_L' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')   
    elif '_R' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')

# drop any columns that persisted after join
print()
print('Removed Columns Peristsing After JOIN:')
for col in df_prep.columns:
    if 'JOIN_' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')
    elif 'Join_' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')
    elif 'Field' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')
    elif '_FID' in col:
        df_prep = df_prep.drop(columns=col)
        print(f'\t{col}')

The following columns intersect as being objects and being null: ['MSAG_LCity' 'MSAG_RCity' 'StPostDir' 'StPreMod'] 

Removed Fully Null columns:
	StPostMod
	StPreMod
	Crm_Cd_4
	StPostDir
	MSAG_LCity
	MSAG_RESN
	MSAG_LESN
	MSAG_RCity

Removed String columns:
	StPreMod
	StPostDir
	MSAG_LCity
	MSAG_RCity
	StPreType
	StArticle
	StPreDir
	Cross_Street
	StPostType
	Weapon_Desc
	Parity_L
	NameCat_L
	Parity_R
	NameCat_R
	StName
	FullName
	Mocodes
	Vict_Descent
	Vict_Sex
	Jurisdiction
	LCity_R
	LCity_L
	Crm_Cd_Desc
	Premis_Desc
	LOCATION
	Status_1
	Status_Desc
	Shape
	AREA_NAME
	Type
	Elevation
	Surface
	Status
	DrivingDir
	Source
	SourceID
	UpdateDate
	Date_Rptd
	DATE_OCC

Removed Latitude and Longtitude Columns
	LAT
	LON

Removed Extraneous Crm_Cd Columns:
	Crm_Cd
	Crm_Cd_1
	Crm_Cd_2
	Crm_Cd_3

Removed Date Columns:
	Date_Disclosed
	Date_Occurred
Removed Victim Sex and Ethnicity Columns:
	Victim_Sex
	Victim_Desc

Removed '_L' and '_R' Columns:
	From_L
	From_R
	To_L
	To_R
	Zip_L
	Shape_Length

In [ ]:
development_cols = df_prep.columns.to_list()
print('Development Data Column List:', '\n')
for x in development_cols:
    print(x)
print()
print('There are', df_prep.shape[0], 'rows and', df_prep.shape[1], 'columns in the' \
    + ' development set.')

Development Data Column List: 

Zip_Code
Accuracy
Crime_Cost
StreetOID
DR_NO
TIME_OCC
AREA
Rpt_Dist_No
Part_1_2
Vict_Age
Premis_Cd
Weapon_Used_Cd
Days_Disclosure
Street_Type
Gender
Crime_Code

There are 183151 rows and 16 columns in the development set.


In [ ]:
# run the python file for checking data types again 
# this time, for the development set
%run data_types.py
data_types(df_prep)

,Column/Variable,Data Type,# of Nulls,Percent Null
0,Zip_Code,float64,0,0.0
1,Accuracy,float64,0,0.0
2,Crime_Cost,float64,0,0.0
3,StreetOID,int64,0,0.0
4,DR_NO,float64,0,0.0
5,TIME_OCC,float64,0,0.0
6,AREA,float64,0,0.0
7,Rpt_Dist_No,float64,0,0.0
8,Part_1_2,float64,0,0.0
9,Vict_Age,float64,0,0.0


In [ ]:
df_prep.head()

,Zip_Code,Accuracy,Crime_Cost,StreetOID,DR_NO,TIME_OCC,AREA,Rpt_Dist_No,Part_1_2,Vict_Age,Premis_Cd,Weapon_Used_Cd,Days_Disclosure,Street_Type,Gender,Crime_Code
OBJECTID,,,,,,,,,,,,,,,,
18,90731.0,50.0,0.0,18,210515199.0,1225.0,5.0,585.0,2.0,55.0,102.0,500.0,0,5,2,0
36,90034.0,50.0,0.0,36,201418966.0,1950.0,14.0,1407.0,2.0,55.0,101.0,400.0,0,6,2,0
37,90034.0,50.0,0.0,36,211423447.0,625.0,14.0,1406.0,2.0,46.0,102.0,400.0,3,6,1,0
44,90033.0,50.0,0.0,43,210410810.0,1845.0,4.0,451.0,1.0,81.0,101.0,400.0,0,8,2,1
56,90011.0,50.0,0.0,55,201300618.0,400.0,13.0,1342.0,2.0,0.0,101.0,102.0,0,6,2,0


In [ ]:
df_ml = df_prep.copy()
df_ml.shape

(183151, 16)

## Train-Test-Validation Split

In [ ]:
# set the random state for reproducibility
rstate = 222

In [ ]:
# Divide train set by .7, test set by .15, and valid set .15
size_train = np.int64(round(.70*len(df_ml),2))
size_valid = np.int64(round(.15*len(df_ml),2))
size_test = np.int64(round(.15*len(df_ml),2))
size_total = size_test + size_valid + size_train

# split the data into the development (train and validation)
# and test set, respectively
train, test = train_test_split(df_ml, train_size=size_train,\
                               random_state=rstate)
valid, test = train_test_split(test, train_size=size_valid,\
                               random_state=rstate)

print('Training size:', size_train)
print('Validation size:', size_valid)
print('Test size:', size_test)
print('Total size:', size_train + size_valid + size_test, '\n')
print('Training percentage:', round(size_train/(size_total),2))
print('Validation percentage:', round(size_valid/(size_total),2))
print('Test percentage:', round(size_test/(size_total),2))

Training size: 128205
Validation size: 27472
Test size: 27472
Total size: 183149 

Training percentage: 0.7
Validation percentage: 0.15
Test percentage: 0.15


In [ ]:
# subset the development and test sets into their own respective dataframes
train_set = train[df_ml.columns]
valid_set = valid[df_ml.columns]
test_set = test[df_ml.columns]

## Write Out the Train, Validation, and Test Sets to Separate .CSV Files

In [ ]:
# write out the train, validation, and test sets to .csv files to the data folder
# such that they are contained in their own unique files
# the train and validation sets can be used without prejudice. However, the data
# can ONLY be run on the test set once
train_set.to_csv(data_path2 + '/train_set.csv')
valid_set.to_csv(data_path2 + '/valid_set.csv')
test_set.to_csv(data_path2 + '/test_set.csv')